In [ ]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import re
import random
import math
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from selenium.webdriver.common.by import By
#오류구문
import warnings
warnings.filterwarnings(action='ignore')
from sqlalchemy import create_engine
import urllib
import pyodbc
import pandas as pd
import numpy as np
import os, sys  


# 데이터베이스 연결
connection_string = (
    "Driver={SQL Server Native Client 11.0};"
    "Server=218.155.23.164;"
    "UID=user1;"
    "PWD=user1!;"
    "Database=capstone;"
)
quoted = urllib.parse.quote_plus(connection_string)
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={quoted}')
#연결 완료

# start crawling
"""크롤링 드라이버 환경설정 
1. headless로 지정
2. no-sandbox로 지정
3. disable-dev-shm-usage로 지정
4. user-agent를 지정"""
webdriver_options = webdriver.ChromeOptions()
webdriver_options.add_argument('--headless')
webdriver_options.add_argument('--no-sandbox')
webdriver_options.add_argument('--disable-dev-shm-usage')
webdriver_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36')

# 크롬드라이버 실행
driver = webdriver.Chrome(ChromeDriverManager().install(), options=webdriver_options)
driver.implicitly_wait(1)
driver.get('https://everytime.kr/login')

# 접속
# driver.find_element_by_name('userid').send_keys('jsh4724')      # 아이디
driver.find_element("name", "userid").send_keys('jsh4724')
# driver.find_element_by_name('password').send_keys('qlalf128') 
driver.find_element("name", "password").send_keys('qlalf128')

## 자유게시판 이동 
# driver.find_element_by_xpath('/html/body/div/form/p[3]/input').click() 
driver.find_element("xpath",'/html/body/div/form/p[3]/input').click() 
time.sleep(2)
# driver.find_element_by_xpath('//*[@id="submenu"]/div/div[2]/ul/li[1]').click()
driver.find_element("xpath",'//*[@id="submenu"]/div/div[2]/ul/li[1]').click()

"""
GUI로부터 값 받아오기
lim_count -> 데이터를 몇 개 가져올 건지 
lim_time  -> 크롤링을 몇 시간 사용할건지
lim_account  -> 계정을 몇 개 사용할거닞
"""
lim_count = 0
lim_time = 0
lim_account = 0
""""데이터프레임 전체를 저장"""
df_all = pd.DataFrame(columns = ['No' ,'Type', 'Time', 'Text', 'crawling start time'])
df_part = pd.DataFrame(columns = ['No' ,'Type', 'Time', 'Text', 'crawling start time'])

"""초기시간 할당"""
start_time = datetime.now()
check_time =  datetime.now()

# while 무한반복문
page_number = 10926 
while True :    
    
    # 데이터프레임 첫 글 작성시간으로부터 1시간이 지나면 데이터프레임을 저장하고 새로운 데이터프레임 할당
    if (math.floor((datetime.now()-check_time).seconds/60)) >= 60 :
        print('60분 초과! start reset time')
        df_all = df_all.append(df_part)
        print('save csv to',str(datetime.now().year).zfill(4)+'년_'+str(datetime.now().month).zfill(2)+'월_'+str(datetime.now().day).zfill(2)+'일_'+str(datetime.now().hour).zfill(2)+'시_'+str(datetime.now().minute).zfill(2)+'분.csv')
        df_part.to_csv(str(datetime.now().year).zfill(4)+'년_'+str(datetime.now().month).zfill(2)+'월_'+str(datetime.now().day).zfill(2)+'일_'+str(datetime.now().hour).zfill(2)+'시_'+str(datetime.now().minute).zfill(2)+'분.csv')
        db_name = str(datetime.now().year).zfill(4)+'년_'+str(datetime.now().month).zfill(2)+'월_'+str(datetime.now().day).zfill(2)+'일_'+str(datetime.now().hour).zfill(2)+'시_'+str(datetime.now().minute).zfill(2)+'분.csv'

        with engine.connect() as cnn:
            df_part.to_sql(db_name,con=cnn, if_exists='replace', index=False)
        print('데이터베이스 저장 완료!!!!!!!!!!!!!')
        df_part = pd.DataFrame(columns = ['No' ,'Type', 'Time', 'Text', 'crawling start time'])
        check_time =  datetime.now()
        
        
    page_number += 1
    driver.get('https://everytime.kr/376864/p/'+ str(page_number))
    driver.implicitly_wait(1)
    all_links = []    
#     posts = driver.find_elements_by_css_selector('article > a.article')
    posts = driver.find_elements(By.CSS_SELECTOR,'article > a.article')
#     posts = driver.findElement(By.cssSelector("article > a.article"))
    links = [post.get_attribute('href') for post in posts]
    count_links = 0 
    for link in links :
        temp = int(str(link.split('/')[-1]))
        if (int(str(link.split('/')[-1])) < temp) :
            print('중복 글입니다! ')
            continue
        count_links += 1
        print(page_number,'번쨰 페이지 ','(', count_links,'/',len(links), ')시작')    
        driver.get(link)
        title = []
        content = []
        title_time = []
        content_time = []
        comment = []
        comment_time = []
    # 제목 : title, 내용 content,  제목 시간 title_time,  내용 시간 content_time, 댓글 comment, 댓글 시간 comment_time
#         titles = driver.find_elements_by_css_selector('div.wrap.articles > article > a > h2')    
#         contents = driver.find_elements_by_css_selector('div.wrap.articles > article > a > p')
#         title_times = driver.find_elements_by_css_selector('div.wrap.articles > article > a > div.profile > time')   
#         content_times = driver.find_elements_by_css_selector('div.wrap.articles > article > a > div.profile > time')   
#         comments = driver.find_elements_by_css_selector('div.wrap.articles > article > div > article.parent > p')
#         comment_times = driver.find_elements_by_css_selector('div.wrap.articles > article > div > article.parent > time')    
        
        titles = driver.find_elements(By.CSS_SELECTOR,'div.wrap.articles > article > a > h2')    
        contents = driver.find_elements(By.CSS_SELECTOR,'div.wrap.articles > article > a > p')
        title_times = driver.find_elements(By.CSS_SELECTOR,'div.wrap.articles > article > a > div.profile > time')  
        content_times = driver.find_elements(By.CSS_SELECTOR,'div.wrap.articles > article > a > div.profile > time')   
        comments = driver.find_elements(By.CSS_SELECTOR,'div.wrap.articles > article > div > article.parent > p')
        comment_times = driver.find_elements(By.CSS_SELECTOR,'div.wrap.articles > article > div > article.parent > time')    

        print('*'*80)
#         print('******start append dataframe(df)******')    
        for data in titles :
            solo_title = data.text
            title.append(data.text)
        for data in contents :
            solo_contents = data.text
            content.append(data.text)
        for data in title_times :
            solo_title_times = data.text
            title_time.append(data.text)
        for data in content_times :
            solo_content_times = data.text
            content_time.append(data.text)
        for data in comments :
            solo_comments = data.text
            comment.append(data.text)
        for data in comment_times :
            solo_comment_times = data.text
            comment_time.append(data.text)

        
        if '분 전' in solo_title_times :
            # 1. 번호 + 제목 + 제목_시간을 데이터프레임에 저장
            df_part = df_part.append({'No': str(link.split('/')[-1])+'001',
                                      'Type': 'title','Time':(datetime.today() - timedelta(hours=0, minutes= int(solo_title_times[:-3]))).strftime('%m/%d %H:%M'),'Text':solo_title, 'crawling start time':datetime.today().strftime('%Y-%m-%d %H:%M')},ignore_index=True)
        else :
            # 1. 번호 + 제목 + 제목_시간을 데이터프레임에 저장
            df_part = df_part.append({'No': str(link.split('/')[-1])+'001',
                                      'Type': 'title','Time':solo_title_times,'Text':solo_title, 'crawling start time':datetime.today().strftime('%Y-%m-%d %H:%M')},ignore_index=True)

        if '분 전' in solo_content_times :
            # 2. 번호 + 내용 + 내용_시간을 데이터프레임에 저장
            df_part = df_part.append({'No': str(link.split('/')[-1])+'002','Type': 'content','Time':(datetime.today() - timedelta(hours=0, minutes= int(solo_content_times[:-3]))).strftime('%m/%d %H:%M'),'Text':solo_contents,'crawling start time':datetime.today().strftime('%Y-%m-%d %H:%M')},ignore_index=True)
        else :
            # 2. 번호 + 내용 + 내용_시간을 데이터프레임에 저장
            df_part = df_part.append({'No': str(link.split('/')[-1])+'002','Type': 'content','Time':solo_content_times,'Text':solo_contents,'crawling start time':datetime.today().strftime('%Y-%m-%d %H:%M')},ignore_index=True)            
            
        # 3. 번호 + 댓글 + 댓글_시간을 데이터프레임에 저장
        count = 0
        count = len(comments)
        print('글번호 ',str(link.split('/')[-1]), '   |   측정 댓글 개수 => ',count, end = '')
        if count != 0 :
            for i in range(count):
                if '분 전' in comment_time[i] :
                    df_part = df_part.append({'No': str(link.split('/')[-1])+str(i+3).zfill(3),'Type': 'comment','Time':(datetime.today() - timedelta(hours=0, minutes= int(comment_time[i][:-3]))).strftime('%m/%d %H:%M'),'Text':comment[i],'crawling start time':datetime.today().strftime('%Y-%m-%d %H:%M')},ignore_index=True)
                else :
                    df_part = df_part.append({'No': str(link.split('/')[-1])+str(i+3).zfill(3),'Type': 'comment','Time':comment_time[i],'Text':comment[i],'crawling start time':datetime.today().strftime('%Y-%m-%d %H:%M')},ignore_index=True)



        print('   |   len(df) : ',len(df_part))    
        print('Time Check !')
        print('진행시간 : ',math.floor((datetime.now()-start_time).seconds/60),'분 ',(datetime.now()-start_time).seconds%60,'초',end = '')
        print('   |   초기화 시간(60분이 될 시 초기화) : ',math.floor((datetime.now()-check_time).seconds/60),'분 ',(datetime.now()-check_time).seconds%60,'초')
        rand = random.randint(3,40)        
        print('지연시간 : ', rand,' 초')
        print('*'*80,'\n')
        time.sleep(rand)
    
    print(page_number,'번쨰 페이지 완료\n\n')
    print('날짜 체크중....')    
    if ((str((datetime.now() - relativedelta(months=1)).month).zfill(2)) == str(solo_title_times[:2])):
        if((str((datetime.now() - relativedelta(months=1)).day).zfill(2)) == str(solo_title_times[3:5])):
            print('30일 초과! End Crawling!')
    print('날짜 체크 완료.\n\n\n')
driver.close()


'2022년_09월_03일_17시_41분.csv'